In [ ]:
import os
import zipfile
import numpy as np
import cv2 as cv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal, Constant
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from google.colab import drive


# Mount Google Drive
drive.mount('/content/drive')

# Path to the zip file
zip_path = '/content/drive/MyDrive/graduation model/standford_beta.zip'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# Path to the dataset folder
dataset_folder = '/content/dataset/standford_beta'

# Function to preprocess images
def preprocess_image(image_path, target_size=(128, 128)):
    image = cv.imread(image_path)
    # Resize image to target size
    image = cv.resize(image, target_size)
    # Normalize pixel values to [0, 1]
    image = image / 255.0
    return image

# Initialize lists to store images and labels
train_images = []
train_labels = []
test_images = []
test_labels = []

# Loop through train and test folders
for split in ['train', 'test']:
    split_path = os.path.join(dataset_folder, split)
    brands = os.listdir(split_path)
    for brand in brands:
        brand_path = os.path.join(split_path, brand)
        image_files = os.listdir(brand_path)
        for image_file in image_files:
            image_path = os.path.join(brand_path, image_file)
            # Preprocess image
            image = preprocess_image(image_path)
            if split == 'train':
                train_images.append(image)
                train_labels.append(brand)
            else:
                test_images.append(image)
                test_labels.append(brand)

# Convert lists to numpy arrays
train_images = np.array(train_images)
test_images = np.array(test_images)

# Perform one-hot encoding for labels
class_labels = {
    'Chevrolet Avalanche Crew Cab 2012': 0,
    'Buick Enclave SUV 2012': 1,
    'BMW X6 SUV 2012': 2,
    'Hyundai Tucson SUV 2012': 3,
    'Chevrolet Corvette ZR1 2012': 4,
    'Chevrolet Camaro Convertible 2012': 5,
    'Mitsubishi Lancer Sedan 2012': 6,
    'Cadillac CTS-V Sedan 2012': 7,
    'HUMMER H3T Crew Cab 2010': 8,
    'Chevrolet Cobalt SS 2010': 9,
    'Chevrolet Tahoe Hybrid SUV 2012': 10,
    'BMW Z4 Convertible 2012': 11,
    'Buick Rainier SUV 2007': 12,
    'Chevrolet Sonic Sedan 2012': 13,
    'Hyundai Accent Sedan 2012': 14,
    'Bugatti Veyron 16.4 Coupe 2009': 15,
    'Buick Verano Sedan 2012': 16,
    'Audi R8 Coupe 2012': 17,
    'Cadillac SRX SUV 2012': 18,
    'Chevrolet Silverado 1500 Extended Cab 2012': 19,
    'Acura TL Sedan 2012': 20,
    'Jeep Liberty SUV 2012': 21,
    'Chevrolet Impala Sedan 2007': 22,
    'Cadillac Escalade EXT Crew Cab 2007': 23,
    'Bentley Continental GT Coupe 2012': 24,
    'Chevrolet HHR SS 2010': 25,
    'Chevrolet Express Van 2007': 26
}

train_labels_encoded = np.array([class_labels[label] for label in train_labels])
test_labels_encoded = np.array([class_labels[label] for label in test_labels])

num_classes = len(class_labels)

model_path = '/content/model.hdf5'
checkpoint = ModelCheckpoint(
    filepath=model_path,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True
)
callbacks = [checkpoint]

# Input shape and layer.
input_shape = (128, 128, 3)
model = Sequential()
model.add(Conv2D(256, (3, 3), padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.2))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.2))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(512, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(216))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization(momentum=0.95, epsilon=0.05,
                             beta_initializer=RandomNormal(mean=0.0, stddev=0.05),
                             gamma_initializer=Constant(value=0.9)))
model.add(Dense(num_classes, activation='softmax'))

initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.9, staircase=True)

opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Prints model summary
model.summary()

# Train the model using train_images and train_labels, and validate using test_images and test_labels
history = model.fit(train_images, train_labels_encoded, epochs=100, validation_data=(test_images, test_labels_encoded), callbacks=callbacks)

# Load best model weights from checkpoint and save results
model.load_weights(model_path)
res = model.evaluate(test_images, batch_size=800)
# Prints results
print("Results")
print(res)


Mounted at /content/drive
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 256)     7168      
                                                                 
 batch_normalization (Batch  (None, 128, 128, 256)     1024      
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 128, 128, 256)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 256)     590080    
                                                                 
 batch_normalization_1 (Bat  (None, 128, 128, 256)     1024      
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


346/346 [==============================] - 132s 216ms/step - loss: 3.3226 - accuracy: 0.0513 - val_loss: 3.6314 - val_accuracy: 0.0452
Epoch 2/100
346/346 [==============================] - ETA: 0s - loss: 3.2149 - accuracy: 0.0765
Epoch 2: val_accuracy improved from 0.04522 to 0.05575, saving model to /content/model.hdf5
346/346 [==============================] - 28s 80ms/step - loss: 3.2149 - accuracy: 0.0765 - val_loss: 3.2821 - val_accuracy: 0.0558
Epoch 3/100
346/346 [==============================] - ETA: 0s - loss: 3.0759 - accuracy: 0.1106
Epoch 3: val_accuracy improved from 0.05575 to 0.08604, saving model to /content/model.hdf5
346/346 [==============================] - 28s 80ms/step - loss: 3.0759 - accuracy: 0.1106 - val_loss: 3.6830 - val_accuracy: 0.0860
Epoch 4/100
346/346 [==============================] - ETA: 0s - loss: 2.9031 - accuracy: 0.1534
Epoch 4: val_accuracy improved from 0.08604 to 0.11414, saving model to /content/model.hdf5
346/346 [=======================